In [1]:
import pandas as pd

df = pd.read_csv("user_data.csv")
df = df.drop(["First Name" , "Last Name" , "City" , "State", "Country" , "Education Level"] , axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475688 entries, 0 to 475687
Data columns (total 25 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Gender                                     475688 non-null  object 
 1   Age                                        475688 non-null  int64  
 2   Annual Vacation Days                       475688 non-null  float64
 3   Average Monthly Spend on Entertainment     475688 non-null  float64
 4   Number of Online Purchases in Last Month   475688 non-null  float64
 5   Number of Charity Donations in Last Year   475688 non-null  float64
 6   Average Weekly Exercise Hours              475688 non-null  float64
 7   Investment Portfolio Value                 475688 non-null  float64
 8   Health Consciousness Rating                475688 non-null  float64
 9   Average Daily Screen Time                  475688 non-null  float64
 10  Environm

In [2]:
# One hot encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in df.columns :
  if df[column].dtype == object :
    df[column] = le.fit_transform(df[column])
    
df

,Gender,Age,Annual Vacation Days,Average Monthly Spend on Entertainment,Number of Online Purchases in Last Month,Number of Charity Donations in Last Year,Average Weekly Exercise Hours,Investment Portfolio Value,Health Consciousness Rating,Average Daily Screen Time,...,Financial Wellness Index,Lifestyle Balance Score,Entertainment Engagement Factor,Social Responsibility Index,Work-Life Balance Indicator,Investment Risk Appetite,Eco-Consciousness Metric,Stress Management Score,Time Management Skill,Lifestyle Choice
0,1,49,30.02,0.00,166.0,3.0,0.00,295.63,0.68,5.86,...,100.00,36.49,2.26,0.0,0.49,0.45,0.0,4.93,44.37,3
1,1,43,23.64,7.23,130.0,0.0,2.80,0.00,0.64,3.52,...,100.00,25.72,0.89,0.0,0.54,3.66,0.0,0.78,1.09,0
2,0,37,7.91,8.09,107.0,0.0,0.00,0.00,0.75,6.74,...,70.42,7.14,1.27,0.0,0.00,7.09,0.0,1.16,33.64,11
3,1,22,20.63,6.39,138.0,0.0,0.00,0.00,0.57,5.38,...,103.17,27.14,1.79,0.0,0.00,1.61,0.0,3.06,100.00,1
4,1,38,19.47,7.80,115.0,0.0,0.07,11.50,5.35,6.00,...,100.00,23.31,1.42,0.0,0.27,1.47,0.0,4.07,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475683,1,34,0.88,43.53,100.0,0.0,0.24,11.12,0.00,3.71,...,100.00,0.98,2.20,0.0,0.77,0.73,0.0,2.90,1.11,4
475684,0,24,8.04,42.33,78.0,0.0,0.00,10.16,0.71,1.67,...,100.00,1.03,2.84,0.0,0.00,6.41,0.0,5.98,7.12,2
475685,0,30,21.44,24.24,189.0,0.0,0.00,0.00,0.88,1.59,...,100.00,14.94,1.07,0.0,0.00,0.01,0.0,0.84,3.62,10
475686,0,62,27.87,45.78,115.0,0.0,0.00,1.61,0.77,7.35,...,0.00,32.25,2.08,0.0,0.32,2.04,0.0,4.27,6.10,6


In [3]:
# โหลดข้อมูล
X = df.drop(["Lifestyle Choice"],axis=1)
y = df["Lifestyle Choice"]

In [4]:
from sklearn.model_selection import train_test_split

X_train , X_temp , y_train , y_temp = train_test_split(X,y, test_size=0.3, random_state=24)
X_val , X_test , y_val , y_test = train_test_split(X_temp , y_temp , test_size=0.4,random_state=24)

In [6]:
X_train.shape[0],X_val.shape[0] , X_test.shape[0]

(332981, 85624, 57083)

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

pipeline = make_pipeline(
    StandardScaler(),
    SGDClassifier(C=100,dual="auto", random_state=24, tol=1e-5,)

)
pipeline

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=100, dual='auto', random_state=24, tol=1e-05))])

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

pipeline = make_pipeline(
    StandardScaler(), 
    LinearSVC(C=100,dual="auto", random_state=24, tol=1e-5)
)
pipeline

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=100, dual='auto', random_state=24, tol=1e-05))])

In [12]:
pipeline.fit(X_train,y_train)
pipeline.score(X_train,y_train)

0.9353746910484382

In [13]:
pipeline.score(X_test,y_test)

0.9352521766550461

In [14]:
from sklearn.model_selection import cross_val_score

accuracy = cross_val_score(pipeline, X,y,cv=10)
accuracy

array([0.93642919, 0.93447413, 0.93674452, 0.93497866, 0.93657634,
       0.9349156 , 0.9369127 , 0.93617692, 0.93592331, 0.93409435])

In [16]:
import numpy as np
np.mean(accuracy)

0.935722571459688

In [38]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.dummy import DummyClassifier


y_pred = pipeline.predict(X_val)

print("Confusion Matrix :")
cm = confusion_matrix(y_val, y_pred)
print(cm)

print("Accuracy Score :")
accuracy_score(y_val, y_pred)

print("Classification Report")
report = classification_report(y_val, y_pred)
print(report)

Confusion Matrix :
[[7008    9    1    4   78    0    1    0    0    0    1    1]
 [   2 6591    0    0    3   12  138    0    9    0  112  190]
 [   0    1 7144    6    0    0    2   19    0   22    0    0]
 [   6    8    6 6963    0    0    0    0   89    0    0    6]
 [  96    0    0    0 7002   13    0    1    0    0    5    0]
 [   4   10    1    1    4 7231   19    5   10    0   22    8]
 [  44   93    6    0    0   50 5824   25   26    0  130  998]
 [   8   92   14    0   28   18  488 5737    0  155  535    8]
 [  65    4    5    0    0    9   96    0 6979    0   10    3]
 [   0    0    5    1    0    0    0   21    0 7034    3    0]
 [  12   64    1    0   18    4   32   32    4    0 6987   22]
 [  14 1227    1   43    9   32  184   31    5    0   82 5442]]
Accuracy Score :
Classification Report
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7103
           1       0.81      0.93      0.87      7057
           2       0.9

### Baselines

#### Random Baseline

In [39]:
random_prediction = np.random.randint(0,11,size=len(y_test))

accuracy = accuracy_score(y_test,random_prediction)
print("Accuracy :",accuracy)


NameError: name 'np' is not defined

#### Zero Rule

In [ ]:
zero_rule_baseline = DummyClassifier(strategy='most_frequent')
zero_rule_baseline.fit(X_train, y_train)


zero_rule_predictions = zero_rule_baseline.predict(X_test)


zero_rule_accuracy = accuracy_score(y_test, zero_rule_predictions)
print("Accuracy :", zero_rule_accuracy)

Accuracy : 0.08286179773312545
